In [1]:
from lib import *
import numpy as np

import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import product

*** THIS NOTEBOOK CAN BE USED TO ANALYSE RAW AND DEC OP.WVFs FROM LARSOFT ***

In [2]:
raw,key_list         = load_larsoft_root("RAW","opdetraw_hist.root",     "opdigiana")
gauss_dec,key_list   = load_larsoft_root("DEC","deconv_gauss_hist.root", "opdecoana")
wiener_dec,key_list  = load_larsoft_root("DEC","deconv_wiener_hist.root","opdecoana")

In [3]:
ordered_raw         = order_wvfs(gauss_dec,key_list)
ordered_gauss_dec   = order_wvfs(gauss_dec,key_list)
ordered_wiener_dec  = order_wvfs(gauss_dec,key_list)

In [4]:
# fig = px.density_contour(x=raw["TRUE"]["PE"],
#                          y=raw["TRUE"]["CH"],
#                          marginal_x="histogram",
#                          marginal_y="histogram",
#                          nbinsx=10*int(np.max(raw["TRUE"]["PE"])),
#                          nbinsy=len(raw["TRUE"]["CH"]))

# fig.update_layout(title="Photon arrival time per channel",
#     xaxis_title='Time in [&mu;s]',
#     yaxis_title='Op. Channel',)
# fig.show()

In [40]:
num = np.random.randint(len(raw["RECO"]["WVF"]))
# print(num)

fig = make_subplots(rows=1, cols=2,
                    # subplot_titles=('Digi wvf', 'Deco wvf')
                    )

fig.add_trace(go.Line(name="RAW #PE: %i"%(raw["RECO"]["PE"][num]),x=raw["RECO"]["WVF_X"][num],y=raw["RECO"]["WVF"][num]-raw["PEDESTAL"]),col=1,row=1)
fig.add_trace(go.Histogram(name="TRUE PHOTONS",x=raw["RECO"]["T0"][num]+raw["PRETRIGGER"]*16e-9*1e6),col=1,row=1)
fig.add_trace(go.Line(name="GAUSS #PE: %.2f"%(gauss_dec["RECO"]["PE"][num]),x=gauss_dec["RECO"]["WVF_X"][num],y=gauss_dec["RECO"]["WVF"][num]),col=2,row=1)
fig.add_trace(go.Line(name="WIENER #PE: %.2f"%(wiener_dec["RECO"]["PE"][num]),x=wiener_dec["RECO"]["WVF_X"][num],y=wiener_dec["RECO"]["WVF"][num]),col=2,row=1)
fig.add_vline(x=gauss_dec["RECO"]["WVF_X"][num][0]+gauss_dec["RECO"]["T0"][num]*16e-9*1e6, line_width=2, line_dash="dash", line_color="gray",col=2)

fig.update_layout(
    title = "Deco. wvf comparison",
    xaxis_title  = "Time in [&mu;s]",
    yaxis_title  = "Amp. in [ADC]",
    xaxis2_title = "Time in [&mu;s]",
    yaxis2_title = "Amp. in [a.u.]",
)
fig.show()

In [6]:
PE_filter = np.asarray(raw["RECO"]["PE"]) != 0
PE_error_gauss = (np.asarray(raw["RECO"]["PE"])[PE_filter]-np.asarray(gauss_dec["RECO"]["PE"])[PE_filter])/np.asarray(raw["RECO"]["PE"])[PE_filter]
PE_error_wiener = (np.asarray(raw["RECO"]["PE"])[PE_filter]-np.asarray(wiener_dec["RECO"]["PE"])[PE_filter])/np.asarray(raw["RECO"]["PE"])[PE_filter]


In [7]:
group_labels = ['GAUSS PHOTON ERROR', 'WIENER PHOTON ERROR']
fig = ff.create_distplot([PE_error_gauss, PE_error_wiener], 
                        group_labels, 
                        curve_type='normal',
                        histnorm="probability",
)

fig.update_layout(title_text='Charge recovery with Normal Distribution')
fig.show()